# OCR Extract Software + Excel creation of data - Jupyter Notebook (Google Colab)
### *Extraction of real-life PDF scanned documents and addition of selected info into an XLSX (Excel) file*
Code made by: ***Victor Hugo Pacheco Padilla***
* The present code is intended to OCR scanned PDFs from variable quality (from good to bad quality), save those files onto a separate folder, and use those new files to extract selected data, and save this info onto an Excel file.

* The code has been tested on both Google Colab platform and Jupyter Lab platform (using in both cases Ubuntu Linux 22.04 LTS - Jammy Jellyfish), but it can be easily adapted to run on Windows or Mac systems, cloud platforms, among others.

* The codes makes use of multiple libraries seen below, to achieve the desired goal for the code.

# Prerequisites

1. **You must install some necessary libraries**, in order to use the TesseractOCR library with the OCRMyPDF library, including the ghostscript library (which might be already installed). The process below allows the installation of the necessary libraries in Ubuntu Linux, and will work with other Debian-based and Ubuntu-based distros. Depending on your OS and package manager, this process will have some differences.

In [ ]:
!sudo apt-get install tesseract-ocr tesseract-ocr-spa ghostscript ocrmypdf

2. **Install the necessary libraries inside Python**. If you use virtual environments or other package manager (such as Conda) the steps may vary.

In [ ]:
pip install pytesseract pypdf2 xlsxwriter

3. Once installed, **import the desired libraries to be used into our environment**. Most of these libraries are Python basics, but if you lack some of these, you should install them in order to run these code. Some of the libraries to be used are:
- PyTesseract: Implementation of Tesseract algorithm inside Python, it is a reliable and tested OCR algorithm which allows us to transcript text from real-life scanned PDFs (and images) with a good grade of precision.
- OCRMyPDF: An implementation of Tesseract focused to make and optimize OCR capability to PDFs, using PyTesseract, adding a layer to already scanned PDFs to make them search readable, and an essencial step to make our code work.
- Pandas: Needed to work with OpenPYXL, in order to save our data to a XLSX file.
- Google Colab - Drive connection (optional): Needed to work with Google Drive files inside Colad. This library **DOES NOT WORK** outside Google Colab.
- os: Needed to work with files inside the OS and make operations with those files, if desired.
- re: Needed to find text inside OCR'd PDFs, using different parameters to find information.
- PyPDF2: Used to retrieve text from OCR'd PDFs and extract found strings to Excel file.

In [3]:
import pytesseract
import ocrmypdf
import pandas as pd
import os
#from google.colab import drive
import openpyxl
import PyPDF2
import re

# Main code

1. *(optional)* Mount the Google Drive drive onto your Google Colab session. This step is only done to retrieve files from Google Drive drive inside Colab, and, as menctioned above, **does not work** on local Jupyter Lab / Python sessions.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. In order to simplify the understand of the code, we define some functions to work with the code:
- **perform_ocr:** Wrapper for OCRMyPDF function, making OCR to every PDF detected inside the desired path (*"pdf_folder"*).
- **extract_text_from_pdf:** Once the OCR process is completed, PyPDF2 allows us to parse text from created PDFs and read it on Python, in order to identify strings based on specific information.
- **extract_string_from_pdf:** Extracts the desired string based on surrounding strings. An optional alternative approach is commented (*"extract_string_from_pdf2"*) to extract strings based on length rather than surrounding strings.

In [ ]:
def perform_ocr(pdf_file_path, output_file_path, language="eng", force_ocr=False, clean=False):
    ocrmypdf.ocr(pdf_file_path, output_file_path, language=language, force_ocr=force_ocr, clean=clean)

def extract_text_from_pdf(pdf_file_path):
    with open(pdf_file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page_text = reader.pages[page_num].extract_text()
            if page_text:
                text += page_text + '\n'  # Add a newline to help separate pages
    return text

def extract_string_from_pdf(text, start_string, end_string):
    # Allow for whitespace and newline characters in between.
    pattern = re.compile(r"(?<=" + re.escape(start_string) + r")[\s\S]*?(?=" + re.escape(end_string) + r")", re.DOTALL)
    match = re.search(pattern, text)
    if match:
        return match.group(0).replace('\n', ' ').replace('\r', '')  # Extracting the string between the substrings
    else:
        return None

# Commented function, for extraction of strings based on length

#def extract_string_from_pdf2(text, start_string2, string_length2):
    # Allow for whitespace and newline characters in between.
    #pattern2 = re.compile(re.escape(start_string2) + r'.{' + str(string_length2) + r'}')
    #match = re.search(pattern2, text)
    #if match:
    #    return match.group(0).replace('\n', ' ').replace('\r', '')[len(start_string2):]  # Extracting the string after the start string
    #else:
    #    return None

# Select the filepath of the PDFs (original and processed)
# Modify the file path depending on your OS and location (default folder structure for Linux)
pdf_folder = r"/home/username/folder/subfolder/PDFS"
output_folder = r"/home/username/folder/subfolder/PDFS_proc"

# Create a list of all PDFs detected inside the folder
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith('.pdf')]

# Define where the identified strings should be. Ideal for legal papers, statements, among others.
start_string = 'Date:'
end_string = "FDA"
# string_length = 20

# Define where the identified strings should be. Ideal for legal papers, statements, among others.
start_string2 = 'Product Type:'
end_string2 = "Reason"
#string_length2 = 16

# Load the existing Excel workbook and worksheet
workbook = openpyxl.Workbook()
worksheet = workbook.active

# Add headers to the worksheet
worksheet['A1'] = 'File Name'
worksheet['B1'] = 'Document Date'
worksheet['C1'] = 'Product Type'

# Set the row counter
row = 2

for pdf_file in pdf_files:
    # Starts OCR inside every PDF
    output_file_name = os.path.basename(pdf_file).replace('.pdf', '_ocr.pdf')
    output_file_path = os.path.join(output_folder, output_file_name)

    perform_ocr(pdf_file, output_file_path, language="eng")

    # Extracts PDF text following the given path above
    pdf_text = extract_text_from_pdf(output_file_path)

    test = extract_string_from_pdf(pdf_text, start_string, end_string)

    #if test is not None:
    #    extracted_string = test + end_string
    #else:
    #    extracted_string = test
    extracted_string = test

    test2 = extract_string_from_pdf(pdf_text, start_string2, end_string2)

    #if test2 is not None:
    #    extracted_string2 = test2 + end_string2
    #else:
    #    extracted_string2 = test2
    extracted_string2 = test2
    
    #extracted_string2 = extract_string_from_pdf2(pdf_text, start_string2, string_length2)

    worksheet.cell(row=row, column=1, value=os.path.basename(pdf_file))
    if extracted_string:
        print(f"The date of the document {pdf_file} is: {extracted_string}")
        worksheet.cell(row=row, column=2, value=extracted_string)
    else:
        print(f"A date was not found for the document {pdf_file}.")

    if extracted_string2:
        print(f"The product type of the document {pdf_file} is: {extracted_string2}")
        worksheet.cell(row=row, column=3, value=extracted_string2)
    else:
        print(f"A product type was not found for the document {pdf_file}.")

    row += 1

# Save the workbook to a new Excel file in a desired path
# Modify the file path depending on your OS and location (default folder structure for Linux)
workbook.save(r"/home/username/folder/subfolder/PDFS_proc/DocInfo.xlsx")

Example PDFs are available on my GitHub repository: https://github.com/VicPachec/ProyectosVicPach/tree/main/PDFS
- You can use **any scanned PDF to test the code**, the code theorically can work with any PDF, configuring the correct language and settings between strings. Depending on the PDF count, size, and scanning quality, results may vary, from some seconds, to less than one hour.
- Future work includes **test with other OCR algorithms** (*Mindee DocTR, Amazon Textract, etc.*), to improve quality, speed, and work with heavier PDFs and poorer scannings.